# Imports

In [1]:
from ragkit import (
    RAGPipeline,
    AugmentationBlock,
    RetrieverBlock,
    RankerBlock,
    TruncateBlock,
    GeneratorBlock,
    LoggingBlock,
    SwitchBlock,
    # Generator,
    HyDE,
    StepBack,
    LeastToMost,
    # BaseAugmentation,
    # LLMToolAugmentation,
    # ChromaRetriever,
    # BaseRetriever,
    # load_docs,
    # Chunker,
    # ChromaIndexer,
    # Embedder,
)

print("✅ Successfully imported ragkit-flow!")


c:\Users\tombj\Documents\Projects\ragkit-api\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Successfully imported ragkit-flow!


In [2]:
from ragkit import ChromaIndexer, ChromaRetriever

indexer = ChromaIndexer(collection_name="dnd")
retriever = ChromaRetriever(indexer=indexer)

📦 Connected to Chroma collection 'dnd' at './chroma_db'


In [3]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# Connect to your LLM endpoint
llm_client = OpenAI(
    base_url="https://a3crnf7woihsvzh3.us-east-1.aws.endpoints.huggingface.cloud/v1/"
)

# Initialise Simple Pipeline

In [4]:
# Example system prompt
system_prompt = (
    "You are a Dungeons & Dragons expert.\n\n"
    "Use the provided context to help answer the question."
    "Context:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"
)

# Build your pipeline
rag = RAGPipeline(
    llm_client=llm_client,
    client_model="tgi",   # or "tgi" if huggingface inference
    system_prompt=system_prompt,
    collection_name="dnd"        # name of your Chroma collection
)

rag.add(AugmentationBlock, augmentations=[HyDE], include_original=True)
rag.add(LoggingBlock, tag="After Augmentations")
rag.add(RetrieverBlock)
rag.add(LoggingBlock, tag="After Retrieval")
rag.add(RankerBlock)
rag.add(TruncateBlock, top_k=3)
rag.add(GeneratorBlock)

# rag.summary()

Embedder initialized on device: cpu
📦 Connected to Chroma collection 'dnd' at './chroma_db'


# Make Query

In [5]:
query = "What are the rules for fireball in 5e?"
result = rag.run(query)

print("\nAnswer:", result["answer"])


──────────────────── 🪵 Pipeline Log — After Augmentations ────────────────────
query: 'What are the rules for fireball in 5e?'
queries: list[2] → ['What are the rules for fireball in 5e?', 'Fireball in 5e']
query_groups: list[2]
  • original → 1 queries
  • hyde_doc → 1 queries
─────────────────────────────────────────────────────────────────────────────


──────────────────── 🪵 Pipeline Log — After Retrieval ────────────────────
query: 'What are the rules for fireball in 5e?'
queries: list[2] → ['What are the rules for fireball in 5e?', 'Fireball in 5e']
query_groups: list[2]
  • original → 1 queries
  • hyde_doc → 1 queries
retrieval_groups: list[2]
  • original → 1 query(s), 5 total hits
  • hyde_doc → 1 query(s), 5 total hits
flattened_results: 10 docs
results: 10 items
─────────────────────────────────────────────────────────────────────────────


Answer: According to the SRD, the rules for a fireball spell in 5e are as follows:

* Level: Sor/Wiz 3
* Components: V, S, M
* Casting